### ***PRE-ELABORAZIONE DATI***

*Nel caso di autogluon la pre-elaborazione è fatta automaticamente dal classificatore; l'unica cosa da fare in modo da rendere fruibile la rete è quella di suddividere il dataset in una parte per il train e una per il test e rendere numerica la variabile target.*

In [ ]:
import pandas as pd
from collections import Counter

# Importazione dataset dal drive
path = # Inserire percorso del file
dataset = pd.read_csv(path)

In [ ]:
from sklearn.model_selection import train_test_split

# Suddivido il dataset esattamente a metà, una per l'addestramento e l'altra per il test
train, test = train_test_split(dataset, test_size=0.5)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder delle due variabili target, quindi da stringhe a interi sia in test che in train e salvo i valori nelle variabili y_test a y_train
# y_test viene utilizzata per la valutazione del modello

# multilabel è il nome della variabile target del dataset, essa naturalemte può variare in base al dataset
y_train = LabelEncoder().fit_transform(train['multilabel'])
y_test = LabelEncoder().fit_transform(test['multilabel'])

# sostituisco i valori originari con la codifica precedentemente fatta
target_index = train.columns.get_loc('multilabel')
train.iloc[:, target_index] = y_train

# elimino la colonna relativa al target nel dataset di test poiché non utile
test = test.drop('multilabel', axis=1)

In [ ]:
# Installazione pacchetti per poter utilizzare Autogluon su Google Colab
!pip install --upgrade pip
!pip install --upgrade setuptools wheel
!pip install --upgrade "mxnet<2.0.0"
!pip install autogluon

### ***MODEL & TRAINING***
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint
import random
from autogluon.tabular import TabularPredictor
import mxnet as mx

label = # Inserire nome della variabile target
train_df = train
test_df = test


from autogluon.tabular import TabularPredictor
# Creazione classificatore 
predictor = TabularPredictor(label=label)
# Addestramento
predictor.fit(train_df, hyperparameters='multimodal')

### ***PREDICTION***
# Utilizzo solo la rete neurale per la predizione 
models = predictor.get_model_names()
nn_name = models[4]

# Predizione 
y_pred = predictor.predict(test_df, model=nn_name)

# Sommario dell'addestramento con le performance dei vari modelli
predictor.fit_summary()

### ***EVALUATION NEURALNET***


In [ ]:
# Matrice di confusione, accuracy, classification_report
from sklearn.metrics import *

# y_test è la variabile che contiene i valori effettivi
# y_pred contiene i valori predetti dal modello
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

acc = accuracy_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
# non presente nella libreria, calcolo mediante formula
f2 = (1+2**2)*((precision*recall)/((2**2*precision)+recall))

In [ ]:
# Multilabel confusione Matrix
mcm = multilabel_confusion_matrix(y_test, y_pred)
print(mcm)

In [ ]:
# Confusione matrix
print(cm)

In [ ]:
# True/False Positive and True/False Negative
FP = cm.sum (axis = 0) - np.diag (cm) 
FN = cm.sum (axis = 1) - np.diag (cm) 
TP = np.diag (cm) 
TN = cm.sum () - (FP + FN + TP)

print('True positive: ', TP)
print('True negative: ', TN)
print('False positive: ', FP)
print('False negative: ', FN)

FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)

print('True positive rate: ', TPR)
print('True negative rate: ', TNR)
print('False positive rate: ', FPR)
print('False negative rate: ', FNR)

In [ ]:
# Stampa report con gli indice della performance
print(report)

In [ ]:
# Stampa di tutte le varie metriche
print('Accuracy: ', acc)
print('Precision_weighted: ', precision)
print('Recall_weighted: ', recall)
print('mcc: ', mcc)
print('f2: ', f2)